In [1]:
import transformers
import torch 
import gc 
import json
from tokens import *

/home/t/tzelilai/Desktop/Thesis/venv/lib64/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
gc.collect()
torch.cuda.empty_cache()

In [3]:
# Load the model and tokenizer with the access token
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    use_auth_token=access_token,
    torch_dtype=torch.bfloat16,
    load_in_4bit=True
)
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=access_token
)

# Create the pipeline with the specified model and tokenizer
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

/home/t/tzelilai/Desktop/Thesis/venv/lib64/python3.9/site-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
2024-11-30 16:26:44.036890: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-30 16:26:45.477842: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-30 16:26:45.966429: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-30 16:26:46.10816

In [4]:
def repeater(messages):

  # Apply chat template to messages
  prompt = pipeline.tokenizer.apply_chat_template(
      messages,
      tokenize=False,
      add_generation_prompt=True
  )

  # Define terminators
  terminators = [
      pipeline.tokenizer.eos_token_id,
  ]

  # Generate text
  outputs = pipeline(
      prompt,
      max_new_tokens=2024*2,
      eos_token_id=terminators,
      do_sample=True,
      temperature=1,
      top_p=0.9,
  )
  return len(prompt), outputs


In [17]:
with open('articles copy.json', 'r') as file: 
    articles = json.load(file)

print(f'{len(articles[len(articles) - 1])} are available.')

5 are available.


In [ ]:
content = ""
for i, article in enumerate(articles[0]):
    content += "Article" + str(i+1) + ":" + article["content"] + '\n\n'

In [31]:
# Define messages
messages = [
{
  "role": "system",
  "content": "You are a journalist tasked with writing a neutral and unbiased news article. You will receive multiple articles on a single topic. Your job is to:"
             "1. Combine the key points from all articles into one cohesive news article."
             "2. The information in the article should flow, without being repeated"
             "3. Ensure the content is factual, informative, and free from bias or personal opinion."
             "4. Provide a comprehensive title for the article"
},                           
    {"role": "user", "content": content}
]

In [32]:
prompt_len, outputs = repeater(messages)
print(outputs[0]["generated_text"][prompt_len:], end='.')

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Title: Musk's Proposed Department of Government Efficiency Raises Concerns About Conflicts of Interest

Cathie Wood, CEO of ARK Invest, believes that Elon Musk will succeed in his new role as head of the Department of Government Efficiency (DOGE), citing his access to proprietary data and experience in streamlining government operations. Wood points to Musk's track record of overcoming incredible odds in his business career, including founding and being pushed out of PayPal in 2000 and revolutionizing space travel and electric vehicles with SpaceX and Tesla.

Musk has been appointed to lead DOGE, a non-governmental body tasked with cutting waste and making Washington work better and faster, alongside Vivek Ramaswamy, a biotech investor and former Republican primary candidate. The department's objectives include scrutinizing federal operations and recommending reforms to reduce waste and improve efficiency.

However, experts are raising concerns about potential conflicts of interest due

In [26]:
# Define messages
messages = [
{
  "role": "system",
  "content": "You are a journalist tasked with writing a neutral and unbiased news article. You will receive multiple articles on a single topic. Your job is to:"
             "1. Combine the key points from all articles into one cohesive news article."
             "2. The information in the article should flow, without being repeated"
             "3. Ensure the content is factual, informative, and free from bias or personal opinion."
             "4. Provide a comprehensive title for the article"
             "4. If differing viewpoints or opinions are observed, present them evenly and objectively."
             "5. Avoid exaggeration or emotionally charged language. Focus on accuracy and clarity."
             "6. Divide the article in paragraphs if necessary "
             "Write in a professional and neutral tone, suitable for publication in a reputable news outlet."
},                           
    {"role": "user", "content": content}
]

In [24]:
prompt_len, outputs = repeater(messages)
print(outputs[0]["generated_text"][prompt_len:], end='.')

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Title: Trump Taps Elon Musk to Lead Department of Government Efficiency, but Questions Remain on Conflicts of Interest

In a surprise move, President-elect Donald Trump has appointed Elon Musk, the CEO of Tesla and SpaceX, to lead the newly created Department of Government Efficiency (DOGE). The agency aims to reduce waste and improve efficiency in the federal government. Musk will be joined by Vivek Ramaswamy, a biotech investor, as co-director of the group.

ARK Invest CEO Cathie Wood, a longtime supporter of Musk and his companies, believes Musk's unique understanding of the current technological landscape and his proprietary data will help him succeed in his new role. Wood pointed out that Musk's companies, including SpaceX, Tesla, and X, possess vast amounts of data, which could be valuable in identifying areas for cost-cutting.

However, critics argue that Musk's vast portfolio of companies, which includes SpaceX, Tesla, and others, creates significant conflicts of interest. Musk

In [27]:
# Define messages
messages = [
    {"role": "system", "content": "You are an AI assistant tasked with producing NEUTRAL news article based on the provided articles."
                                  "Take in consideration all the provided articles and write a new article as a journalist, ensuring the content" 
                                  "is unbiased, factual, and informative."},
                                
    {"role": "user", "content": content}
]


In [30]:
prompt_len, outputs = repeater(messages)
print(outputs[0]["generated_text"][prompt_len:], end='.')

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Title: Trump Taps Elon Musk to Lead Department of Government Efficiency, Amid Concerns Over Conflicts of Interest

President-elect Donald Trump has appointed Elon Musk, the CEO of Tesla and SpaceX, to lead a newly created Department of Government Efficiency (DOGE), a non-governmental body tasked with slashing government spending and streamlining federal agencies. While Cathie Wood, CEO of ARK Invest, backs Musk's appointment, citing his ability to bring proprietary data to the table, others are raising concerns over potential conflicts of interest.

Musk's companies, which include SpaceX, Tesla, and Neuralink, have received billions of dollars in federal contracts over the past decade, and he has been vocal about his desire to reduce regulations and cut government spending. However, critics argue that his appointment could create conflicts of interest, as he may be in a position to benefit from cost-cutting measures that affect his own companies.

The concerns are not new, as Trump pre

In [29]:
# Define messages
messages = [
    {"role": "system", "content": "You are an AI assistant tasked with producing a NEUTRAL and UNBIASED news article based on the provided articles summaries. "
                                  "Your goal is to analyze all the provided information and write a single cohesive news article as a professional journalist. "
                                  "The content must be factual, concise, and informative. Do not include personal opinions or assumptions. "
                                  "Ensure the tone is formal and journalistic, suitable for a wide audience. "
                                  "Provide a clear and appropriate title for the article."},
                                
    {"role": "user", "content": content}
]


In [50]:
prompt_len, outputs = repeater(messages)
print(outputs[0]["generated_text"][prompt_len:], end='.')

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Title: Department of Government Efficiency to Tackle Waste and Improve Efficiency, but Concerns Over Conflicts of Interest Remain

A newly announced initiative aims to reduce waste and improve efficiency in the federal government, led by billionaire Elon Musk and Vivek Ramaswamy, as part of the Department of Government Efficiency (DOGE). The non-governmental body's objectives are to streamline operations and cut waste, with Musk bringing his expertise in innovation and data analysis to the table. Cathie Wood, CEO of Ark Invest, believes Musk's companies, including SpaceX and Tesla, possess valuable insights that can inform more efficient government spending.

The DOGE plans to leverage Musk's expertise to identify areas of inefficiency and recommend solutions. However, critics have raised concerns about potential conflicts of interest, as Musk's companies operate in industries regulated by federal agencies, and his close relationship with President-elect Donald Trump may compromise his

In [51]:
# Define messages
messages = [
    {
        "role": "system", 
        "content": (
            "You are an AI assistant tasked with analyzing multiple article summaries provided by  different sources. "
            "Being an neutral Journalist, write a structured, NEUTRAL, and FACTUAL news article that clearly presents the event or topic. "
            "Ensure the new produced article remains unbiased and informative, giving balanced attention to each perspective. "
            "Provide a clear and concise title and present different points of view IF they are spotted on the different articles."
        )
    },
    {"role": "user", "content": content}
]


In [52]:
prompt_len, outputs = repeater(messages)
print(outputs[0]["generated_text"][prompt_len:], end='.')

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


**Title:** "Musk-Leaded Department of Government Efficiency Faces Criticism Over Potential Conflicts of Interest"

A new initiative aimed at reducing government waste and improving efficiency has been announced, with entrepreneur Elon Musk and Vivek Ramaswamy at its helm. The Department of Government Efficiency (DOGE) aims to streamline federal operations and cut unnecessary spending, leveraging the expertise of Musk's companies, including SpaceX and Tesla. However, the appointment has raised concerns about potential conflicts of interest, as Musk's companies operate in areas regulated by federal agencies and he may benefit from his own recommendations.

Ark Invest CEO Cathie Wood, a longtime supporter of Musk, believes his unique understanding of the technological landscape and control over vast amounts of proprietary data will bring a fresh perspective to government efficiency initiatives. Wood notes Musk's track record of overcoming challenges in business and his ability to bring in

In [29]:
with open('generated_article.json', 'w') as file: 
    json.dump([outputs[0]["generated_text"][prompt_len:]], file, indent=1)

In [12]:
gc.collect()
torch.cuda.empty_cache()

In [13]:
del outputs

In [14]:
del model